In [1]:
import numpy as np

 **What is the goal** : to produce a contextual embedding using the attention mechanism 
 
 Input : sequence of embeddings (at first we will fix the size for a sequence size of 10)
 
 Workflow : We generate three other vectors for each vector of the sequence (query, key and value )
 
 How to generate the query, key and value matrix? 
 
 We use three trained linear neural works which map the the sequence vectors from the embedding space to the (query|key|value) space  :: we will do that using keras or pytorch 
 
 After generating the query vector for a let's say a vector, we generate the key and value values  for all the vectors
 
 We calculate the alignment between the query and the key vectors, and generate the soft scores using softmax(QK/sqrt(100)) 
 
 After we multiply the value matrix with the soft scores and here we are ! 
 

### 1- Creating the Query, Key and Value neural networks:

In [2]:
import torch
import torch.nn as nn

# Define the linear neural network class
class LinearNet(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        out = self.linear(x)
        return out

In [3]:
input_size = 300  # embedding size
output_size = 100 

QueryNet = LinearNet(input_size, output_size)
KeyNet = LinearNet(input_size, output_size)
ValueNet = LinearNet(input_size, output_size)

In [7]:
QueryNet(torch.randn(1, input_size))

tensor([[-0.2013,  0.3086, -0.3140,  0.4095, -0.1363,  0.7736, -0.7541, -0.5053,
          0.7333,  0.5436, -0.2293, -0.3904,  0.0727,  0.2498,  0.2504, -0.0299,
          0.8427,  0.1175, -0.5161,  0.1756,  0.4146, -1.2297,  0.3067,  0.8257,
          0.1042, -0.8673,  0.3044,  0.7660,  0.4154, -0.7564, -1.3894,  0.5866,
          0.1456,  0.0402, -1.1214,  0.0050, -0.3385, -0.8557, -1.0842, -0.6060,
         -0.6634,  0.5292, -1.1790, -1.2778,  0.0941, -0.1970,  0.0395,  0.2002,
         -0.2107, -0.5291, -0.3964, -0.8224, -0.1421,  0.9500, -0.1042,  0.5344,
         -0.6678,  1.0328, -0.0245, -1.0527,  0.4336,  0.1805,  0.8529,  1.6139,
         -0.3068,  1.1001, -0.1137,  0.5524,  0.5909,  0.5677, -0.8780,  0.4706,
         -0.2418, -0.3641,  0.3176,  0.4458,  0.4062,  0.2241, -0.6790,  0.0603,
          0.0516,  1.1742,  0.1403, -0.8218, -0.1120, -0.3468,  0.4971, -0.0737,
          0.4384, -0.7292,  0.5264, -0.0719,  0.4676,  0.2461, -0.0116,  0.3718,
          0.5335,  0.0394,  

### 2- Contextualize embedding

In [20]:
import torch.nn.functional as F

def contextualize_word(word_embedding, sequence_embedding):
    # word_embedding = red ; sequence_embedding = This a red dog
    
    query_vector =  QueryNet(word_embedding) # We assume vector is a float tensor of size 300 
    key_matrix = KeyNet(sequence_embedding) # Must be of shape seq_len * 300
    value_matrix = ValueNet(sequence_embedding) 
    
    scores = (query_vector @ key_matrix.T) / np.sqrt(100)
    
    soft_scores = F.softmax(scores, dim=-1)

    context_embedding = soft_scores @ value_matrix
    
    return context_embedding

**Testing the function**

In [21]:
word_embedding = torch.randn(1, 300)
sequence_embedding = torch.randn(4, 300)

In [22]:
contextualize_word(word_embedding, sequence_embedding)

tensor([[ 0.3287,  0.4841, -0.0669, -0.2088,  0.0806,  0.0531,  0.2906, -0.0368,
          0.1630, -0.3398,  0.2978, -0.2344,  0.2401,  0.3502,  0.9361, -0.1037,
         -0.0410, -0.2895,  0.1485, -0.2414,  0.0217, -0.2183, -0.0505, -0.2789,
         -0.3176, -0.3250,  0.1926, -0.0934,  0.6953, -0.1575,  0.1577, -0.3216,
         -0.0292, -0.1829,  0.1994,  0.6841,  0.3014, -0.1067,  0.2029,  0.2907,
         -0.5654, -0.9305,  0.2001, -0.4611, -0.1033, -0.1310, -0.0889, -0.6519,
          0.6422, -0.1379, -0.1439, -0.0844, -0.1376,  0.2105,  0.4158,  0.1122,
          0.1666,  0.2599,  0.1154,  0.5296,  0.3872,  0.1615, -0.1404, -0.0223,
          0.0612,  0.1443,  0.3555, -0.0133, -0.0178,  0.0318,  0.1221, -0.1122,
         -0.6418,  0.3554,  0.0369,  0.0235,  0.1442,  0.0797, -0.0616,  0.2121,
          0.1668,  0.0259, -0.2411,  0.1392, -0.2285,  0.1335,  0.1379, -0.0668,
         -0.1689, -0.0714,  0.5075,  0.1196,  0.2058,  0.2444, -0.1720, -0.6190,
         -0.0596,  0.6516,  

### 3-  Getting the embeddings using word2vec: 

### 4 - How to train the Q, K and V neural networks? 

For that we will need to use a downstream task, like optimizing them for which task ? Which task will tell us whether our embeddings are good or not ? 

For this, we are going to use MLM (Mask Language Modelling) in order to train the q, k and v neural networks. 

Example : 
     By giving the sentence : This is a [MASK] dog, the model is expected to generate candidat words for the MASK token
     
We first need to create a tokenizer ! let's do it ! 

In [ ]:
masked_token_embeddings = torch.randn(768)

In [ ]:
def MLM(sequence, idx_mask):
    masked_word = sequenc# Define the linear neural network class
class mlm_head(nn.Module):
    def __init__(self, embedding_size):
        super(mlm_head, self).__init__()
        self.transform = nn.Linear(embedding_size, embedding_size)

    def forward(self, x):
        out = self.transform(x)
        return oute[idx_mask]
    pass

In [ ]:
def MLM(sequence, idx_mask):
    
    masked_word = sequence[idx_mask]
    sequence[idx_mask] = masked_token_embeddings # mask the embedding of the hidden word
    
    contextualized_mask = contextualize_word(masked_token_embeddings, sequence)
    
    predicted_embedding = mlm_head(contextualized_mask)
    
    return predicted_embedding 

# Now we gotta take the closest vector to the predicted_embedding in our vocabulary
# It seems like the decoding phase
# Another approach is to contextualize every word, and then use a head nn to predict the masked word ?? it seems weirder